In [20]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import itertools

In [164]:
### version1 addframe, FFT###
def addframe(num, grayvalue, img): # for FFT calculation
    # add pixel
    num_insert = int((num-1)/2)
    # create block
    bk_tb = np.zeros((num_insert,img.shape[1]),np.uint8)
    bk_lr = np.zeros((img.shape[0]+(num_insert*2),num_insert),np.uint8)
    # decide gray value
    bk_tb[:,:] = grayvalue # white
    bk_lr[:,:] = grayvalue # white
    # insert pixel (top, bottom)
    array = np.insert(img, 0, bk_tb, axis=0)
    array = np.insert(array, array.shape[0], bk_tb, axis=0)

    # insert pixel (left, right)
    array = np.insert(array, [0], bk_lr, axis=1)
    array = np.insert(array, [array.shape[1]], bk_lr, axis=1)
    
    return np.array(array)

def FFT(img, num):
    amp = []
#     amp_u = []
#     amp_d = []
#     amp_l = []
#     amp_r = []
    phase = []
#     phase_u = []
#     phase_d = []
#     phase_l = []
#     phase_r = []
    num_insert = int((num-1)/2)
    h = img.shape[0] - (num_insert*2)
    w = img.shape[1] - (num_insert*2)
    for  y in range(num_insert,img.shape[0]-num_insert):
        for x in range(num_insert,img.shape[1]-num_insert):
            img1 = img[(y-num_insert):(y+num_insert+1),(x-num_insert):(x+num_insert+1)]
            fimg = np.fft.fft2(img1)
#             fshift = np.fft.fftshift(fimg)
            a = np.abs(fimg)
            p = np.angle(fimg)
#             a = np.abs(fshift)
#             p = np.angle(fshift)
            amp.append(a[num_insert, num_insert]) # centre coordinates (e.g. 5X5([0,0]~[4,4]) ->[2,2])
#             amp_u.append(a[num_insert-1,num_insert])
#             amp_d.append(a[num_insert+1,num_insert])
#             amp_l.append(a[num_insert,num_insert-1])
#             amp_r.append(a[num_insert,num_insert+1])
            phase.append(p[num_insert, num_insert]) # centre coordinates (e.g. 5X5([0,0]~[4,4]) ->[2,2])
#             phase_u.append(p[num_insert-1, num_insert])
#             phase_d.append(p[num_insert+1, num_insert])
#             phase_l.append(p[num_insert, num_insert-1])
#             phase_r.append(p[num_insert, num_insert+1])
    amp = np.reshape(amp,(h,w))
#     amp_u = np.reshape(amp_u,(h,w))
#     amp_d = np.reshape(amp_d,(h,w))
#     amp_l = np.reshape(amp_l,(h,w))
#     amp_r = np.reshape(amp_r,(h,w))
    phase = np.reshape(phase,(h,w))
#     phase_u = np.reshape(phase_u,(h,w))
#     phase_d = np.reshape(phase_d,(h,w))
#     phase_l = np.reshape(phase_l,(h,w))
#     phase_r = np.reshape(phase_r,(h,w))
    return amp, phase
#     return amp, phase, amp_u, amp_d, amp_l, amp_r, phase_u, phase_d, phase_l, phase_r
#############################

### update addframe, FFT###
def addframe_update(num, grayvalue, img): # for FFT calculation in case of NXN
    # add pixel
    num_insert = int(num/2) 
    # create block (top:num_insert, bottom:num_insert+1, left:num_insert, right:num_insert+1)
    bk_t = np.zeros((num_insert-1,img.shape[1]),np.uint8)
    bk_b = np.zeros((num_insert,img.shape[1]),np.uint8)
    bk_l = np.zeros((img.shape[0]+(num_insert*2)-1,num_insert),np.uint8)
    bk_r = np.zeros((img.shape[0]+(num_insert*2)-1,num_insert-1),np.uint8)
    # decide gray value
    bk_t[:,:] = grayvalue # white
    bk_b[:,:] = grayvalue # white
    bk_l[:,:] = grayvalue # white
    bk_r[:,:] = grayvalue # white
    # insert pixel (top, bottom)
    array = np.insert(img, 0, bk_t, axis=0)
    array = np.insert(array, array.shape[0], bk_b, axis=0)

    # insert pixel (left, right)
    array = np.insert(array, [0], bk_l, axis=1)
    array = np.insert(array, [array.shape[1]], bk_r, axis=1)
    
    return np.array(array)


def FFT_update(img, num):
#Note num = more than 4 and only 2^n
    amp_l = []
    amp_r = []
    phase_l = []
    phase_r = []
    num_insert = int(num/2)-1
    h = img.shape[0] - (num_insert*2)-1
    w = img.shape[1] - (num_insert*2)-1
    for  y in range(num_insert,img.shape[0]-num_insert-1):
        for x in range(num_insert,img.shape[1]-num_insert-1):
            img1 = img[(y-num_insert):(y+num_insert+2),(x-num_insert):(x+num_insert+2)]
            h1,w1 = img1.shape[0],img1.shape[1]
            img_comp = []
#### for 1/4 compression e.g. 32->8, 12->3
            for i in range(0,h1,4):
                for j in range(0,w1,4):
                    img_comp.append(np.average(img1[i:i+4,j:j+4]))
            img_comp = np.reshape(img_comp,(int(h1/4),int(w1/4)))
#### for 1/2 compression e.g. 32->16, 12->6
#             for i in range(0,h1,2):
#                 for j in range(0,w1,2):
#                     img_comp.append(np.average(img1[i:i+2,j:j+2]))
#             img_comp = np.reshape(img_comp,(int(h1/2),int(w1/2)))
###########################################
#                     for k in range(i,i+2):
#                         for l in range(j,j+2):
#                             ave =img[k][l].mean()
            fimg = np.fft.fft2(img_comp)
            a = np.abs(fimg)
            p = np.angle(fimg)
#### for 1/4 compression e.g. 32->8, 12->3
            amp_l.append(a[int(h1/8)-1, int(w1/8)-1]) # centre coordinates left side (e.g. 5X5([0,0]~[7,7]) ->[3,3])
            amp_r.append(a[int(h1/8), int(w1/8)]) # centre coordinates right side (e.g. 5X5([0,0]~[7,7]) ->[4,4])
            phase_l.append(p[int(h1/8)-1, int(w1/8)-1]) # centre coordinates left side (e.g. 5X5([0,0]~[7,7]) ->[3,3])
            phase_r.append(p[int(h1/8), int(w1/8)]) # centre coordinates right side (e.g. 5X5([0,0]~[7,7]) ->[4,4])
#### for 1/2 compression e.g. 32->16, 12->6
#             amp_l.append(a[int(h1/4)-1, int(w1/4)-1]) # centre coordinates left side (e.g. 5X5([0,0]~[7,7]) ->[3,3])
#             amp_r.append(a[int(h1/4), int(w1/4)]) # centre coordinates right side (e.g. 5X5([0,0]~[7,7]) ->[4,4])
#             phase_l.append(p[int(h1/4)-1, int(w1/4)-1]) # centre coordinates left side (e.g. 5X5([0,0]~[7,7]) ->[3,3])
#             phase_r.append(p[int(h1/4), int(w1/4)]) # centre coordinates right side (e.g. 5X5([0,0]~[7,7]) ->[4,4])
###########################################
    amp_l = np.reshape(amp_l,(h,w))
    amp_r = np.reshape(amp_r,(h,w))
    phase_l = np.reshape(phase_l,(h,w))
    phase_r = np.reshape(phase_r,(h,w))
    return amp_l, phase_l, amp_r, phase_r
###########################

### detect edge ###
# position 0: white 1: part of shading and edge 2: part of shading and not edge 
#                   3: part of symbol and edge  4: part of symbol and not edge 
def detect_edge(array,shading,txt,txt_ad):
    edge =[]
    num = 1
    for y in range(num,array.shape[0]-num):
        for x in range(num,array.shape[1]-num):
            if array[y,x] == 255:
                edge.append(0)
            elif (array[y,x] == 0) and (array[y,x] == shading[y-num,x-num]) and (array[y,x] != txt[y-num,x-num]):
                total_count = 0
                for i,j in itertools.product(range(-1,2),range(-1,2)):
                    if array[y+i,x+j] == 255:
                        total_count += 1
                if total_count >= 1:
                    edge.append(1)
                else:
                    edge.append(2)
            elif (array[y,x] == 0) and (array[y,x] != shading[y-num,x-num]) and (array[y,x] == txt[y-num,x-num]):
                total_count1 = 0
                for k,l in itertools.product(range(-1,2),range(-1,2)):
                    if txt_ad[y+k,x+l] == 255:
                        total_count1 += 1
                if total_count1 >= 1:
                    edge.append(3)
                else:
                    edge.append(4)
            else:
                total_count2 = 0
                for m,n in itertools.product(range(-1,2),range(-1,2)):
                    if txt_ad[y+m,x+n] == 255:
#                     if array[y+m,x+n] == 255:
                        total_count2 += 1
                if total_count2 >= 1:
                    edge.append(3)
                else:
                    edge.append(4)
    return edge
##################

# # the number of dimension applying FFT
# N = 3 #(N * N)
# # output(test)
# img_combined_out = './sample/data1combined_ex.png'
# cv2.imwrite(img_combined_out, np.array(array))


In [ ]:
tic()
import os
# assign the area applying FFT
# num = 31 # ver.1 num * num (e.g. 31X31(16+1+16)) 
num = 32 # num * num (e.g. 32X32(15+2+15))
ck_gv_grid = 3 # check gray value 3X3 grid
os.makedirs('./output'+str(num)+'_8_csv',exist_ok=True) # create directory
number_of_data = 200
for i in range(number_of_data):
    # load image
    img_shade = cv2.imread('../test/output/data'+str(i+1)+'shading.png')
    img_text = cv2.imread('../test/output/data'+str(i+1)+'text.png')
    img_combined = cv2.imread('../test/output/data'+str(i+1)+'combined.png')
#     img_shade = cv2.imread('./sample/data1shading.png')
#     img_text = cv2.imread('./sample/data1text.png')
#     img_combined = cv2.imread('./sample/data1combined.png')

    # get information per pixel 
    gray_shade = cv2.cvtColor(img_shade,cv2.COLOR_RGB2GRAY)
    gray_text = cv2.cvtColor(img_text,cv2.COLOR_RGB2GRAY)
    gray_combined = cv2.cvtColor(img_combined,cv2.COLOR_RGB2GRAY)


    # add frame for FFT
#     gray_combined_ex = addframe(num,255,gray_combined)
#     amp,phase = FFT(gray_combined_ex, num)
# #     amp,phase, amp_u, amp_d, amp_l, amp_r, phase_u, phase_d, phase_l, phase_r = FFT(gray_combined_ex, num)

    # Update add frame for FFT
    gray_combined_ex = addframe_update(num,255,gray_combined)
    amp_l,phase_l,amp_r, phase_r = FFT_update(gray_combined_ex, num)
    
    # add frame for distance to the edge ## update add predictors
    gray_combined_ad = addframe(ck_gv_grid,255,gray_combined)
    gray_text_ad = addframe(ck_gv_grid,255,gray_text)
    edge = detect_edge(gray_combined_ad, gray_shade, gray_text, gray_text_ad)
    edge = np.reshape(edge, (gray_combined.shape[0],gray_combined.shape[1]))

    # get image size
    height, width = gray_combined.shape[:2]

    # initialise
    data =[]
### add FFT value as predictors ###
#     for y in range(height):
#         for x in range(width):
#             if gray_combined[y,x] == 255:
#                 data.append([y,x,gray_combined[y,x],amp[(y*width)+x],phase[(y*width)+x],0])
#             elif (gray_combined[y,x] == 0) & (gray_combined[y,x] == gray_shade[y,x]) & (gray_combined[y,x] != gray_text[y,x]):
#                 data.append([y,x,gray_combined[y,x],amp[(y*width)+x],phase[(y*width)+x],1])
#             elif (gray_combined[y,x] == 0) & (gray_combined[y,x] != gray_shade[y,x]) & (gray_combined[y,x] == gray_text[y,x]):
#                 data.append([y,x,gray_combined[y,x],amp[(y*width)+x],phase[(y*width)+x],2])
#             else:
#                 data.append([y,x,gray_combined[y,x],amp[(y*width)+x],phase[(y*width)+x],2])
####################################

### PGM update add FFT value, distance to the edge as predictors ###
# label 0: white 1: part of shading 2: part of symbol
    for y in range(height):
        for x in range(width):
            if gray_combined[y,x] == 255:
                data.append([y,x,gray_combined[y,x],amp_l[y,x],amp_r[y,x],phase_l[y,x],phase_r[y,x],edge[y,x],0])
#                 data.append([y,x,gray_combined[y,x],amp[y,x],phase[y,x],edge[y,x],0])
#                 data.append([y,x,gray_combined[y,x],amp[y,x],amp_u[y,x],amp_d[y,x],amp_l[y,x],amp_r[y,x],phase[y,x],phase_u[y,x],phase_d[y,x],phase_l[y,x],phase_r[y,x],edge[y,x],0])
            elif (gray_combined[y,x] == 0) and (gray_combined[y,x] == gray_shade[y,x]) and (gray_combined[y,x] != gray_text[y,x]):
                data.append([y,x,gray_combined[y,x],amp_l[y,x],amp_r[y,x],phase_l[y,x],phase_r[y,x],edge[y,x],1])
#                 data.append([y,x,gray_combined[y,x],amp[y,x],phase[y,x],edge[y,x],1])
#                 data.append([y,x,gray_combined[y,x],amp[y,x],amp_u[y,x],amp_d[y,x],amp_l[y,x],amp_r[y,x],phase[y,x],phase_u[y,x],phase_d[y,x],phase_l[y,x],phase_r[y,x],edge[y,x],1])
            elif (gray_combined[y,x] == 0) and (gray_combined[y,x] != gray_shade[y,x]) and (gray_combined[y,x] == gray_text[y,x]):
                data.append([y,x,gray_combined[y,x],amp_l[y,x],amp_r[y,x],phase_l[y,x],phase_r[y,x],edge[y,x],2])
#                 data.append([y,x,gray_combined[y,x],amp[y,x],phase[y,x],edge[y,x],2])
#                 data.append([y,x,gray_combined[y,x],amp[y,x],amp_u[y,x],amp_d[y,x],amp_l[y,x],amp_r[y,x],phase[y,x],phase_u[y,x],phase_d[y,x],phase_l[y,x],phase_r[y,x],edge[y,x],2])
            else:
                data.append([y,x,gray_combined[y,x],amp_l[y,x],amp_r[y,x],phase_l[y,x],phase_r[y,x],edge[y,x],2])
#                 data.append([y,x,gray_combined[y,x],amp[y,x],phase[y,x],edge[y,x],2])
#                 data.append([y,x,gray_combined[y,x],amp[y,x],amp_u[y,x],amp_d[y,x],amp_l[y,x],amp_r[y,x],phase[y,x],phase_u[y,x],phase_d[y,x],phase_l[y,x],phase_r[y,x],edge[y,x],2])
####################################################################

    temp = np.array(data)
    df = pd.DataFrame(temp, columns =['y', 'x','gray_value','amp_l','amp_r','phase_l','phase_r','position', 'label'])
#     df = pd.DataFrame(temp, columns =['y', 'x','gray_value','amp','amp_u','amp_d','amp_l','amp_r','phase','phase_u','phase_d','phase_l','phase_r','position', 'label'])
#     df.to_excel('./output_xlsx/data'+str(i+1)+'.xlsx', index=None)
    df.to_csv('./output'+str(num)+'_8_csv/data'+str(i+1)+'.csv', index=False,)
    print('Create data ' +str(i+1))
toc()



Create data 1


In [3]:
# for counting time
import time

def tic():
    #require  to import time
    global start_time_tictoc
    start_time_tictoc = time.time()


def toc(tag="elapsed time"):
    if "start_time_tictoc" in globals():
        print("{}: {:.9f} [sec]".format(tag, time.time() - start_time_tictoc))
    else:
        print("tic has not been called")


In [162]:
### FFT update unit test###
num = 32
img = cv2.imread('../test/output/data1combined.png',0)
img_ex = addframe_update(num,255,img)
amp_l,phase_l,amp_r, phase_r = FFT_update(img_ex, num)
# img_ex.shape


KeyboardInterrupt: 

In [158]:
amp_l.shape

(140, 930)

In [136]:
phase_l

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [65]:
#### FFT unit test ####
import itertools
num = 5
img = cv2.imread('../test/output/data1combined.png',0)
img_ex = addframe(num,255,img)
amp = []
phase = []
num_insert = 2 #int((num-1)/2)
for  y in range(num_insert,img_ex.shape[0]-num_insert):
    for x in range(num_insert,img_ex.shape[1]-num_insert):
        img1 = img_ex[(y-num_insert):(y+num_insert+1),(x-num_insert):(x+num_insert+1)]
        fimg = np.fft.fft2(img1)
        #fshift = np.fft.fftshift(fimg)
        a = np.abs(fimg)
        p = np.angle(fimg)
#         print(y,x)
#         print(fimg)
#         print(a)
#         print(img1.shape)
        for i,j in itertools.product(range(5),range(5)):
            amp.append(a[i,j])
            phase.append(p[i,j])
#             amp.append(a[num_insert, num_insert])
#             phase.append(p[num_insert, num_insert])
# amp = np.reshape(amp,(img.shape[0],img.shape[1]))
# amp.shape

3255000

In [45]:
#### detect distance unittest ####
# num = 3
# img1 = cv2.imread('../test/output/data1combined.png',0)
# img2 = cv2.imread('../test/output/data1shading.png',0)
# img3 = cv2.imread('../test/output/data1text.png',0)
# img_ex = addframe(num,255,img1)
# img_ex_txt = addframe(num,255,img3)
# edge = detect_edge(img_ex,img2,img3,img_ex_txt)
# # amp = np.reshape(amp,(img.shape[0],img.shape[1]))
# edge = np.reshape(edge,(img1.shape[0],img.shape[1]))
# np.savetxt('./test_edge_detect.csv',edge, fmt ='%d', delimiter=',')